In [1]:
import ROOT
ROOT.gSystem.Load("libDelphes")
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
from array import array

Welcome to JupyROOT 6.13/01


In [2]:
lumi = 25000.

In [3]:
f1 = ROOT.TFile.Open("/Users/vince/work/Madgraph/MG5_aMC_v2_6_1/Hgg_RUN/Events/run_01/tag_1_delphes_events.root")

In [4]:
t1 = f1.Get("Delphes")

In [5]:
ROOT.gROOT.ProcessLine(
"struct MyStruct {\
   Float_t gamma_lead_pt;\
   Float_t gamma_lead_eta;\
   Float_t gamma_lead_phi;\
   Float_t gamma_sublead_pt;\
   Float_t gamma_sublead_eta;\
   Float_t gamma_sublead_phi;\
   Int_t n_photons;\
   Int_t n_jets;\
   Int_t n_leptons;\
   Float_t lead_isolation;\
   Float_t sublead_isolation;\
   Float_t ht;\
   Float_t weight;\
   Char_t    fMyCode[10];\
};" );

In [6]:
from ROOT import MyStruct

mystruct = MyStruct()

sig_file = ROOT.TFile("HggSignal.root","RECREATE")
signal_tree = ROOT.TTree( 'Tree', 'Higgs Events' )
signal_tree.Branch( 'mypts', mystruct, 'gamma_lead_pt/F:gamma_lead_eta:gamma_lead_phi:gamma_sublead_pt:gamma_sublead_eta:gamma_sublead_phi:n_photons/I:n_jets:n_leptons:lead_isolation/F:sublead_isolation:ht:weight/F' )
signal_tree.Branch( 'mycode', ROOT.AddressOf( mystruct, 'fMyCode' ), 'MyCode/C' )
 
for event in t1:
    mystruct.n_photons = event.Photon.GetEntries()
    #mystruct.weight = event.Weight
    mystruct.weight = event.Event[0].Weight
    mystruct.ht = event.ScalarHT[0].HT
    for i, gamma in enumerate(event.Photon):
        if i == 0: 
            l = gamma.P4()
            mystruct.gamma_lead_pt = l.Pt()
            mystruct.gamma_lead_eta = l.Eta()
            mystruct.gamma_lead_phi = l.Phi()
            mystruct.lead_isolation = gamma.IsolationVar
        elif i == 1: 
            l = gamma.P4()
            mystruct.gamma_sublead_pt = l.Pt()
            mystruct.gamma_sublead_eta = l.Eta()
            mystruct.gamma_sublead_phi = l.Phi()
    n_jets = 0
    for jet in event.Jet:
        if jet.P4().Pt() > 40: n_jets +=1
    mystruct.n_jets = n_jets
    n_leptons = 0
    for electron in event.Electron:
        if electron.P4().Pt()> 10:
            n_leptons += 1
    for muon in event.Muon:
        if muon.P4().Pt() > 3:
            n_leptons += 1
    mystruct.n_leptons = n_leptons
    for i in range(10):
        mystruct.fMyCode   = "%03d" % i      # note string assignment
    signal_tree.Fill()

In [7]:
signal = signal_tree.CloneTree() 
sig_file.Write()
sig_file.Close()

In [8]:
f2 = ROOT.TFile.Open("/Users/vince/work/Madgraph/MG5_aMC_v2_6_1/HggBgk_RUN/Events/run_06/tag_1_delphes_events.root")
t2 = f2.Get("Delphes")

In [9]:
mystruct = MyStruct()
 
bkg_file = ROOT.TFile("HggBackground.root","RECREATE")
background_tree = ROOT.TTree( 'Tree', 'Background Events' )
background_tree.Branch( 'mypts', mystruct, 'gamma_lead_pt/F:gamma_lead_eta:gamma_lead_phi:gamma_sublead_pt:gamma_sublead_eta:gamma_sublead_phi:n_photons/I:n_jets:n_leptons:lead_isolation/F:sublead_isolation:ht:weight/F' )
background_tree.Branch( 'mycode', ROOT.AddressOf( mystruct, 'fMyCode' ), 'MyCode/C' )
 
for event in t2:
    mystruct.n_photons = event.Photon.GetEntries()
    mystruct.weight = event.Event[0].Weight
    mystruct.ht = event.ScalarHT[0].HT
    for i, gamma in enumerate(event.Photon):
        if i == 0: 
            l = gamma.P4()
            mystruct.gamma_lead_pt = l.Pt()
            mystruct.gamma_lead_eta = l.Eta()
            mystruct.gamma_lead_phi = l.Phi()
            mystruct.lead_isolation = gamma.IsolationVar
        elif i == 1: 
            l = gamma.P4()
            mystruct.gamma_sublead_pt = l.Pt()
            mystruct.gamma_sublead_eta = l.Eta()
            mystruct.gamma_sublead_phi = l.Phi()
            mystruct.sublead_isolation = gamma.IsolationVar
    n_jets = 0
    for jet in event.Jet:
        if jet.P4().Pt() > 40: n_jets +=1
    mystruct.n_jets = n_jets
    n_leptons = 0
    for electron in event.Electron:
        if electron.P4().Pt()> 10:
            n_leptons += 1
    for muon in event.Muon:
        if muon.P4().Pt() > 3:
            n_leptons += 1

    mystruct.n_leptons = n_leptons
    for i in range(10):
        mystruct.fMyCode   = "%03d" % i      # note string assignment
    background_tree.Fill()

In [10]:
background = background_tree.CloneTree() 
bkg_file.Write()
bkg_file.Close()